You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.


# Exercises



In [12]:
import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from env import user, password, host, get_db_url

# Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

# Read the case, department, and source data into their own spark dataframes.



In [13]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/10/22 05:12:46 WARN Utils: Your hostname, shorters-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.93 instead (on interface en0)
22/10/22 05:12:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 05:12:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/22 05:12:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/22 05:12:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/22 05:12:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [8]:
url = get_db_url('311_data')
all_query = """
select *
from source
join cases 
using (source_id)
join dept 
using(dept_division)
"""
cases_query = 'select * from cases'
dept_query = 'select * from dept'
source_query = 'select * from source'

In [5]:
cases_df = pd.read_sql(cases_query, url)
dept_df = pd.read_sql(dept_query, url)
source_df = pd.read_sql(source_query, url)

In [14]:
cases = spark.createDataFrame(cases_df)
dept = spark.createDataFrame(dept_df)
sources = spark.createDataFrame(source_df)

# Let's see how writing to the local disk works in spark:

## Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [15]:
import os

In [17]:
sources.write.json('data/sources_json', mode='overwrite')

In [20]:
sources.show()

+-----+---------+--------------------+
|index|source_id|     source_username|
+-----+---------+--------------------+
|    0|   100137|    Merlene Blodgett|
|    1|   103582|         Carmen Cura|
|    2|   106463|     Richard Sanchez|
|    3|   119403|      Betty De Hoyos|
|    4|   119555|      Socorro Quiara|
|    5|   119868| Michelle San Miguel|
|    6|   120752|      Eva T. Kleiber|
|    7|   124405|           Lori Lara|
|    8|   132408|       Leonard Silva|
|    9|   135723|        Amy Cardenas|
|   10|   136202|    Michelle Urrutia|
|   11|   136979|      Leticia Garcia|
|   12|   137943|    Pamela K. Baccus|
|   13|   138605|        Marisa Ozuna|
|   14|   138650|      Kimberly Green|
|   15|   138650|Kimberly Green-Woods|
|   16|   138793| Guadalupe Rodriguez|
|   17|   138810|       Tawona Martin|
|   18|   139342|     Jessica Mendoza|
|   19|   139344|        Isis Mendoza|
+-----+---------+--------------------+
only showing top 20 rows



In [22]:
(
    sources.write.format('csv')
    .mode('overwrite')
    .option('header', 'True')
    .save('data/sources_csv')
)

## Inspect your folder structure. What do you notice?
* Files split into numerous parts, both visible and non.

In [23]:
json_files = os.listdir('data/mpg_json/')
json_files

['.part-00002-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 'part-00002-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '.part-00001-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 'part-00007-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '._SUCCESS.crc',
 '.part-00006-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 'part-00005-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 'part-00000-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '.part-00005-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 'part-00001-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '.part-00003-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 'part-00004-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '.part-00000-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 '_SUCCESS',
 'part-00006-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json',
 '.part-00007-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',
 '.part-00004-c4b24cd5-e4f5-4614-80ed-4f86e15af48b-c000.json.crc',

# Inspect the data in your dataframes. 
    Are the data types appropriate? 
    Write the code necessary to cast the values to the appropriate types.



## Are the data types appropriate?
* Not all of them
* Changes:
    - datetime < -- > case opened, case closed, sla due date
    - binary < -- > case late, case closed
    - string < -- > council_district


In [28]:
from pprint import pprint

In [30]:
for subset in (cases, dept, sources):
    pprint(subset.show(1, vertical=True))
    pprint('---')

22/10/22 06:11:46 WARN TaskSetManager: Stage 4 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/22 06:11:51 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 18): Attempting to kill Python Worker


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row

None
'---'
-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row

None
'---'
-RECORD 0-----------

In [29]:
for subset in (cases, dept, sources):
    pprint(subset.dtypes)
    pprint('---')

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]
'---'
[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]
'---'
[('index', 'bigint'), ('source_id', 'string'), ('source_username', 'string')]
'---'


## Write the code necessary to cast the values to the appropriate types

In [37]:
#start with booleans --> change
# for subset in ('case_late', 'case_closed'):
#     cases.select(subset).distinct().show()

# cases.withColumn('case_closed',
#                 expr('case_closed == "YES"')
#                 ).select('case_closed').distinct().show()

for subset in ('case_late', 'case_closed'):
    cases = cases.withColumn(f'{subset}',
            expr(f'{subset} == "YES"')
                    )

# cases.select('case_late').show(5)

In [45]:
#convert date times

# for subset in ('case_opened_date', 'case_closed_date', 'SLA_due_date'):
#     cases.select(subset).show(1)

for subset in ('case_opened_date', 'case_closed_date', 'SLA_due_date'):
    cases = cases.withColumn(f'{subset}', to_timestamp(subset, 'M/d/yy H:m'))
    #use --> .select(f'{subset}').show() --> to view results before assignment

In [50]:
cases = cases.withColumn('council_district', col('council_district').cast('string'))

# How old is the latest (overdue: in terms of days past SLA) currently open issue? 


In [86]:
#first find currently open issues --> need case_opened_date, num_days_late, case_closed
(
    cases.select('num_days_late', 'SLA_days', 'case_opened_date')
         .where('case_closed == False')
         .where('num_days_late > 0')
         .where('num_days_late != "NaN"') # --> NaN is string here, so requires double quotes
         .sort('num_days_late', ascending=False)
         .show(9)
)

22/10/23 05:34:10 WARN TaskSetManager: Stage 60 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------+-------------------+
|     num_days_late|   SLA_days|   case_opened_date|
+------------------+-----------+-------------------+
|       348.6458333|15.77247685|2017-01-01 13:57:00|
|       348.6458333|15.77859954|2017-01-01 13:48:00|
|348.52356480000003|       15.0|2017-01-02 11:26:00|
|347.58256939999995|       15.0|2017-01-03 10:01:00|
|       345.3894213|       15.0|2017-01-05 14:39:00|
|       341.6458333|9.899560185|2017-01-14 10:54:00|
|       341.4006481|        7.0|2017-01-17 14:23:00|
|       340.6083333|       15.0|2017-01-10 09:24:00|
|       339.6458333|15.58215278|2017-01-10 18:31:00|
+------------------+-----------+-------------------+
only showing top 9 rows



# How long has the oldest (in terms of days since opened) currently opened issue been open?


# How many Stray Animal cases are there?
* 26760

In [95]:
#cases.select('service_request_type').show(1, vertical=True)
(
    cases.select('service_request_type')
        .where('service_request_type == "Stray Animal"')
        .groupBy('service_request_type')
        .count()
        .show()
)

22/10/23 06:01:15 WARN TaskSetManager: Stage 79 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



# How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
* 113902



In [97]:
(
    cases.select('service_request_type')
        .where('dept_division == "Field Operations"')
        .where('service_request_type != "Officer Standby"')
        .count()
)

22/10/23 06:03:25 WARN TaskSetManager: Stage 82 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


113902

# Convert the council_district column to a string column. --> Converted in work above

# Extract the year from the case_closed_date column.


In [99]:
cases.show(1, vertical=True)

22/10/23 06:51:42 WARN TaskSetManager: Stage 85 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/23 06:51:46 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 85 (TID 324): Attempting to kill Python Worker
-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [117]:
#remember to put the slash before the d
cases = cases.withColumn('year', regexp_extract('case_closed_date', r'^\d+' , 0))#.select('year').show(1)

# Convert num_days_late from days to hours in new columns num_hours_late.



In [120]:
cases = cases.withColumn('num_hours_late', expr('num_days_late * 24'))#.select('num_hours_late').show()

# Join the case data with the source and department data.

In [121]:
# sources.show(1, vertical=True)
# dept.show(1, vertical=True)
# cases.show(1, vertical=True)
cases = cases.join(dept, 'dept_division', 'left')

-RECORD 0---------------------------
 index           | 0                
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [127]:
cases = cases.join(sources, 'source_id', 'left')

# Are there any cases that do not have a request source?

* no



In [130]:
cases.filter(cases.source_id.isNull()).show()

22/10/23 07:27:03 WARN TaskSetManager: Stage 113 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+---------+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--------+-----------+---------------+----------------+----+--------------+---------+----------------------+-------------------+-----+---------------+
|source_id|dept_division|case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|service_request_type|SLA_days|case_status|request_address|council_district|year|num_hours_late|dept_name|standardized_dept_name|dept_subject_to_SLA|index|source_username|
+---------+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--------+-----------+---------------+----------------+----+--------------+---------+----------------------+-------------------+-----+---------------+
+---------+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--

# What are the top 10 service request types in terms of number of requests?



In [133]:
(
    cases.select('service_request_type')
        .groupBy('service_request_type')
        .count()
        .sort('count', ascending=False)
        .show(10)
)

22/10/23 07:29:47 WARN TaskSetManager: Stage 140 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



# What are the top 10 service request types in terms of average days late?


In [135]:
(
    cases.select('service_request_type', 'num_days_late')
        .groupBy('service_request_type')
        .avg()
        .sort('avg(num_days_late)', ascending=False)
        .show(10)
)

22/10/23 07:32:18 WARN TaskSetManager: Stage 152 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|service_request_type|avg(num_days_late)|
+--------------------+------------------+
|Request for Resea...|               NaN|
|CCO_Request for R...|               NaN|
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...| 153.9972403942857|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|  135.928516124798|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
+--------------------+------------------+
only showing top 10 rows



# Does number of days late depend on department?


In [138]:
(
    cases.select('dept_division', 'num_days_late')
        .groupBy('dept_division')
        .avg()
        .sort('avg(num_days_late)', ascending=False)
        .show(10)
)

22/10/23 07:50:43 WARN TaskSetManager: Stage 173 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|       dept_division|avg(num_days_late)|
+--------------------+------------------+
|          District 6|               NaN|
|          District 8|               NaN|
|          District 2|               NaN|
|          District 3|               NaN|
|          District 1|               NaN|
|         District 10|               NaN|
|          District 7|               NaN|
|          District 9|               NaN|
|Code Enforcement ...|135.92851612479794|
|        Reservations|       66.03116319|
+--------------------+------------------+
only showing top 10 rows



# How do number of days late depend on department and request type?

In [140]:
(
    cases.select('dept_division', 'service_request_type','num_days_late')
        .groupBy('dept_division')
        .avg()
        .sort('avg(num_days_late)', ascending=False)
        .show(10)
)

22/10/23 07:52:01 WARN TaskSetManager: Stage 191 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|       dept_division|avg(num_days_late)|
+--------------------+------------------+
|          District 2|               NaN|
|          District 8|               NaN|
|         District 10|               NaN|
|          District 7|               NaN|
|          District 3|               NaN|
|          District 1|               NaN|
|          District 6|               NaN|
|          District 9|               NaN|
|Code Enforcement ...|135.92851612479794|
|        Reservations|       66.03116319|
+--------------------+------------------+
only showing top 10 rows

